In [ ]:
import pandas as pd
import glob

# Find all matching files and take the first one

files = glob.glob("SVRM3 Drive data/REXCURVE*.csv")[0]
df = pd.read_csv(first_file, skiprows=6, engine='python', encoding='utf-8')

print(df.shape)        # number of rows and columns
print(df.columns)      # actual column names Pandas detected

(31170, 69)
Index(['0', '25.744', '-4508.972', '97.5', '553.279', '4632.783', '-104.3',
       '102.2289', '-4519.558', '90.8', '339.8514', '4516.84', '-85.4',
       '0.6733', '316.9441', '6.6', '327.8701', '266.3756', '2.1', '243.0025',
       '0.1', '-1.5', '242.9166', '-0.3469', '0.6', '23.1638', '0.286', '51.4',
       '22.6311', '0.2', '41.7', '24.9415', '0.286.1', '9.5', '20.4063',
       '0.858', '34.6', '360', '8191', '4786', '5145', '57337', '2', '1.65',
       '1.65.1', '-1.04', '-0.61', '311', '8', '268', '3', '2.527', '0.7905',
       '1.744', '1.91', '1', '267.1', '267.1003', '1.663', '1.682', '1.652',
       '1.726', '1.74', '1.714', '191.9004', '148.3002', '203.6998',
       '160.2001', 'Unnamed: 68'],
      dtype='object')


In [45]:
import pandas as pd
import os

# Folder containing your CSV or Excel files
folder_path = "SVRM3 Drive data"

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    # Read CSV while skipping first 6 rows
    df = pd.read_csv(file_path, skiprows=6)
    # Save the cleaned file (overwrite or save to a new folder)
    df.to_csv(file_path, index=False)


PermissionError: [Errno 13] Permission denied: 'SVRM3 Drive data\\REXCURVE 2025-09-11 073726 825.CSV'

In [ ]:
csv_files = glob.glob("*.CSV")
print(f"Found {len(csv_files)} CSV files")

results = []

for file in csv_files:
    try:
        # Read only the first few lines to inspect the new header/data
        with open(file, "r", encoding="utf-8", errors="ignore") as f:
            first_lines = [next(f).strip() for _ in range(3)]
        
        # Load first line through pandas as a quick sanity check
        df_head = pd.read_csv(file, nrows=1, low_memory=False)
        n_cols = df_head.shape[1]

        results.append((file, "OK", n_cols, first_lines[0]))
    except Exception as e:
        results.append((file, f"Error: {e}", None, None))

print("\nVerification summary:")
for name, status, ncols, header in results:
    print(f"{name:50s} | {status:10s} | columns: {ncols} | first line: {header[:60] if header else 'N/A'}")

Found 1 CSV files

Verification summary:
processed_file.csv                                 | Error: No columns to parse from file | columns: None | first line: N/A


In [22]:
hor_ver = df.iloc[:, 43:47]
hor_ver.columns = ["vert", "hor", "vert_offset", "hor_offset"]
print(hor_ver.head())

   vert   hor  vert_offset  hor_offset
0  1.65  1.65        -1.04       -0.61
1  1.65  1.65        -1.04       -0.61
2  1.65  1.65        -1.04       -0.61
3  1.65  1.65        -1.04       -0.61
4  1.65  1.65        -1.04       -0.61


In [23]:
# total positions
# vert_total and hor_total give the true position including offsets.
# Useful to detect misalignment, drift, or tool wear.
hor_ver['vert_total'] = hor_ver['vert'] + hor_ver['vert_offset']
hor_ver['hor_total'] = hor_ver['hor'] + hor_ver['hor_offset']

# differences
# vert_diff and hor_diff show deviation from expected positions.
# Growing differences over time indicate machine drift or calibration issues.
hor_ver['vert_diff'] = hor_ver['vert'] - hor_ver['vert_offset']
hor_ver['hor_diff'] = hor_ver['hor'] - hor_ver['hor_offset']

# Rate-of-change (first derivative)
# vert_change and hor_change detect sudden movements or spikes between time steps.
# Useful for spotting mechanical issues, vibrations, or transient faults.
hor_ver['vert_change'] = hor_ver['vert'].diff()
hor_ver['hor_change'] = hor_ver['hor'].diff()
hor_ver['vert_offset_change'] = hor_ver['vert_offset'].diff()
hor_ver['hor_offset_change'] = hor_ver['hor_offset'].diff()

# Rolling statistics
# vert_ma5 / hor_ma5 smooth data to reveal gradual trends, like tool wear.
# vert_std5 / hor_std5 detect instability or increased variability, which may precede failures.
window = 5
hor_ver['vert_ma5'] = hor_ver['vert'].rolling(window).mean()
hor_ver['vert_std5'] = hor_ver['vert'].rolling(window).std()
hor_ver['hor_ma5'] = hor_ver['hor'].rolling(window).mean()
hor_ver['hor_std5'] = hor_ver['hor'].rolling(window).std()

# Offset ratios ===
# vert_offset_ratio and hor_offset_ratio measure relative size of offset to actual movement.
# High ratios indicate significant deviations or calibration problems.
hor_ver['vert_offset_ratio'] = hor_ver['vert_offset'] / (hor_ver['vert'] + 1e-6)
hor_ver['hor_offset_ratio'] = hor_ver['hor_offset'] / (hor_ver['hor'] + 1e-6)

# === Step 7: Fill NaNs from diff/rolling ===
# Fill NaNs that appear from diff() or rolling calculations with 0
hor_ver.fillna(0, inplace=True)

# === Step 8: Preview feature-rich DataFrame ===
print(hor_ver.head())

   vert   hor  vert_offset  hor_offset  vert_total  hor_total  vert_diff  \
0  1.65  1.65        -1.04       -0.61        0.61       1.04       2.69   
1  1.65  1.65        -1.04       -0.61        0.61       1.04       2.69   
2  1.65  1.65        -1.04       -0.61        0.61       1.04       2.69   
3  1.65  1.65        -1.04       -0.61        0.61       1.04       2.69   
4  1.65  1.65        -1.04       -0.61        0.61       1.04       2.69   

   hor_diff  vert_change  hor_change  vert_offset_change  hor_offset_change  \
0      2.26          0.0         0.0                 0.0                0.0   
1      2.26          0.0         0.0                 0.0                0.0   
2      2.26          0.0         0.0                 0.0                0.0   
3      2.26          0.0         0.0                 0.0                0.0   
4      2.26          0.0         0.0                 0.0                0.0   

   vert_ma5  vert_std5  hor_ma5  hor_std5  vert_offset_ratio  hor_of

In [33]:
df_names = pd.read_csv(first_file, engine='python', encoding='utf-8')

In [34]:
df_names

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,09/03/2025 08:28:05.567 AM,Unnamed: 1
TIME,A-0-0100,A-0-0102,A-0-0110,A-0-0100,A-0-0102,A-0-0110,A-0-0100,A-0-0102,A-0-0110,A-0-0100,A-0-0102,A-0-0110,A-0-0100,A-0-0102,A-0-0110,A-0-0100,A-0-0102,A-0-0110,A-0-0100,A-0-0102,A-0-0110,A-0-0100,A-0-0102,A-0-0110,A-0-0100,A-0-0102,A-0-0110,A-0-0100,A-0-0102,A-0-0110,A-0-0100,A-0-0102,A-0-0110,A-0-0100,A-0-0102,A-0-0110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-,22,22,22,24,24,24,26,26,26,28,28,28,12,12,12,18,18,18,10,10,10,16,16,16,14,14,14,15,15,15,20,20,20,21,21,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
NaN,Actual position value,Actual velocity value,Actual torque/force value,Actual position value,Actual velocity value,Actual torque/force value,Actual position value,Actual velocity value,Actual torque/force value,Actual position value,Actual velocity value,Actual torque/force value,Actual position value,Actual velocity value,Actual torque/force value,Actual position value,Actual velocity value,Actual torque/force value,Actual position value,Actual velocity value,Actual torque/force value,Actual position value,Actual velocity value,Actual torque/force value,Actual position value,Actual velocity value,Actual torque/force value,Actual position value,Actual velocity value,Actual torque/force value,Actual position value,Actual velocity value,Actual torque/force value,Actual position value,Actual velocity value,Actual torque/force value,UserVarGlobal.MLD1_MeetUnit_Hor_M2,UserVarGlobal.MLD1_MeetUnit_Vert_M1,UserVarGlobal.MLD2_MeetUnit_Hor_M3,UserVarGlobal.MLD2_MeetUnit_Vert_M4,UserVarGlobal.TotaalAantalSpaken,HMI.StangData.d_stang,HMI.StangData.d_vert,HMI.StangData.d_hor,HMI.StangData.d_vert_offset,HMI.StangData.d_hor_offset,HMI.StangData.Ls,HMI.StangData.L1,HMI.StangData.L2,HMI.StangData.ActiefType,HMI.MeetUnitSettings.MLD1_WP1_ActualValue,HMI.MeetUnitSettings.MLD1_WP2_ActualValue,HMI.MeetUnitSettings.MLD2_WP1_ActualValue,HMI.MeetUnitSettings.MLD2_WP2_ActualValue,HMI.StangHistorie[1].Goedgekeurd,HMI.StangHistorie[1].NummerMeetcyclus,HMI.StangHistorie[1].L_v,HMI.StangHistorie[1].d_gem,HMI.StangHistorie[1].d_max,HMI.StangHistorie[1].d_min,HMI.StangHistorie[1].b_gem,HMI.StangHistorie[1].b_max,HMI.StangHistorie[1].b_min,HMI.Sta...Positie_BeginVerdunningVert,HMI.Sta...Positie_EindVerdunningVert,HMI.Sta...Positie_BeginVerdunningHor,HMI.Sta...Positie_EindVerdunningHor,NaN
ms,mm,mm/min,%,mm,mm/min,%,mm,mm/min,%,mm,mm/min,%,Deg,1/min,%,Deg,1/min,%,Deg,1/min,%,Deg,1/min,%,mm,mm/min,%,mm,mm/min,%,mm,mm/min,%,mm,mm/min,%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
uint,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,int,int,int,int,uint,float,float,float,float,float,float,float,float,int,float,float,float,float,uint,float,float,float,float,float,float,float,float,float,float,float,float,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82440,25.5115,-9.584,-6.6,553.487,12.088,4.2,102.0147,-12.589,-6.5,340.0893,7.725,12.1,307.5034,316.7868,4.9,279.4358,266.4471,1.6,243.0011,-0.0501,-1.9,242.9852,-0.0107,1.3,23.1638,0.286,51.9,22.6311,0.572,41,24.9415,0.286,9.4,20.4063,0.572,34.4,359,8191,4756,5110,57395,2,1.65,1.65,-1.04,-0.61,311,8,268,3,2.527,0.7895,1.737,1.902,1,267.2001,267.2001,1.664,1.681,1.654,1.725,1.74,1.703,192.1004,148.3001,203.8997,160.3001,NaN
82442,25.5112,-10.443,-7,553.4874,11.301,4.6,102.0143,-13.018,-6.9,340.0896,8.44,12.3,311.302

In [42]:
names = df_names.iloc[2]
names

09/03/2025 08:28:05.567 AM    HMI.Sta...Positie_EindVerdunningHor
Unnamed: 1                                                    NaN
Name: (nan, Actual position value, Actual velocity value, Actual torque/force value, Actual position value, Actual velocity value, Actual torque/force value, Actual position value, Actual velocity value, Actual torque/force value, Actual position value, Actual velocity value, Actual torque/force value, Actual position value, Actual velocity value, Actual torque/force value, Actual position value, Actual velocity value, Actual torque/force value, Actual position value, Actual velocity value, Actual torque/force value, Actual position value, Actual velocity value, Actual torque/force value, Actual position value, Actual velocity value, Actual torque/force value, Actual position value, Actual velocity value, Actual torque/force value, Actual position value, Actual velocity value, Actual torque/force value, Actual position value, Actual velocity value, Actual 

In [43]:
# Remove NaN and join all names into a single string
import math

flat_string = ", ".join(str(c) for c in names if not (isinstance(c, float) and math.isnan(c)))

print(flat_string)


HMI.Sta...Positie_EindVerdunningHor
